In [23]:
# ขั้นตอนที่ 1 ติดตั้งไลบรารีที่จำเป็น

In [24]:
!pip install osmnx==1.9.1 folium -q

In [25]:
# ขั้นตอนที่ 2 Import และตั้งค่าพื้นฐาน

In [26]:
import osmnx as ox
import networkx as nx
import folium

In [27]:
ox.settings.use_cache = True
ox.settings.log_console = False

In [28]:
yourplace = (13.810925413203364, 100.03999535873761)
silpakorn = (13.817015070841382, 100.04208567514407)

In [29]:
# ขั้นตอนที่ 3 ดึงข้อมูลแผนที่พื้นที่กว้าง (กรุงเทพฯ และ นครปฐม)

In [30]:
north = max(yourplace[0], silpakorn[0]) + 0.01
south = min(yourplace[0], silpakorn[0]) - 0.01
east = max(yourplace[1], silpakorn[1]) + 0.01
west = min(yourplace[1], silpakorn[1]) - 0.01

G = ox.graph_from_bbox(north, south, east, west, network_type='drive')

/tmp/ipython-input-30-1862867417.py:6: FutureWarning: The `north`, `south`, `east`, and `west` parameters are deprecated and will be removed in the v2.0.0 release. Use the `bbox` parameter instead.
  G = ox.graph_from_bbox(north, south, east, west, network_type='drive')


In [31]:
# หา node ที่ใกล้ที่สุดกับ yourplace และ silpakorn

In [32]:
orig_node = ox.distance.nearest_nodes(G, yourplace[1], yourplace[0])
dest_node = ox.distance.nearest_nodes(G, silpakorn[1], silpakorn[0])

In [33]:
# เติมข้อมูลความเร็วเฉลี่ยบนถนน ถ้าไม่มีข้อมูล maxspeed กำหนด default 40 กม./ชม.

In [34]:
for u, v, k, data in G.edges(keys=True, data=True):
    if 'maxspeed' in data:
        if isinstance(data['maxspeed'], list):
            speed_str = data['maxspeed'][0]
        else:
            speed_str = data['maxspeed']
        try:
            speed = float(speed_str.split()[0])
        except:
            speed = 40.0
    else:
        speed = 40.0
    data['speed_kph'] = speed
    # คำนวณเวลาขับขี่เป็นวินาที
    data['travel_time'] = data['length'] / (speed * 1000 / 3600)

In [35]:
# หาเส้นทางที่ระยะทางสั้นที่สุด (weight='length')
route_length = sum(ox.utils_graph.get_route_edge_attributes(G, route, 'length'))
route_travel_time = sum(ox.utils_graph.get_route_edge_attributes(G, route, 'travel_time'))

print(f"Shortest route length: {route_length:.2f} meters")
print(f"Estimated travel time: {route_travel_time/60:.2f} minutes")


Shortest route length: 1037.51 meters
Estimated travel time: 1.56 minutes


/tmp/ipython-input-35-1117023988.py:2: FutureWarning: The `get_route_edge_attributes` function has been deprecated and will be removed in the v2.0.0 release. Use the `route_to_gdf` function instead.
  route_length = sum(ox.utils_graph.get_route_edge_attributes(G, route, 'length'))
/tmp/ipython-input-35-1117023988.py:3: FutureWarning: The `get_route_edge_attributes` function has been deprecated and will be removed in the v2.0.0 release. Use the `route_to_gdf` function instead.
  route_travel_time = sum(ox.utils_graph.get_route_edge_attributes(G, route, 'travel_time'))


In [36]:
# ถ้าต้องการพล็อตเส้นทางบนแผนที่แบบ interactive ใช้ folium
route_map = ox.plot_route_folium(G, route, route_color='red')

/tmp/ipython-input-36-3845288638.py:2: FutureWarning: The `folium` module has been deprecated and will be removed in the v2.0.0 release. You can generate and explore interactive web maps of graph nodes, edges, and/or routes automatically using GeoPandas.GeoDataFrame.explore instead, for example like: `ox.graph_to_gdfs(G, nodes=False).explore()`. See the OSMnx examples gallery for complete details and demonstrations.
  route_map = ox.plot_route_folium(G, route, route_color='red')


In [37]:
# เพิ่ม marker จุดเริ่มต้น
folium.Marker(location=yourplace, popup="Your Place", icon=folium.Icon(color='green')).add_to(route_map)

In [38]:
# เพิ่ม marker จุดปลายทาง
folium.Marker(location=silpakorn, popup="Silpakorn University", icon=folium.Icon(color='red')).add_to(route_map)

In [39]:
# แสดงแผนที่ interactive
route_map